In [ ]:
!pip install fairseq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 85.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.7/273.7 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11170777 sha256=612265b3c602c64eaa34160c773eb6f1d9f578f2c84b0b4ae1df8e642f14fcb1
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Cr

In [ ]:
# get the roberta-large-mnli model
import torch
import json
from datetime import datetime
from fairseq.data.data_utils import collate_tokens

roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')

from google.colab import drive
drive.mount('/content/drive')

Downloading: "https://github.com/pytorch/fairseq/zipball/main" to /root/.cache/torch/hub/main.zip
100%|██████████| 751652118/751652118 [00:09<00:00, 80823260.50B/s] 
1042301B [00:00, 42754024.37B/s]
456318B [00:00, 20032199.25B/s]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


label_map = {0: 'contradiction', 1: 'neutral', 2: 'entailment', 3: ""}
ncorrect, nsamples = 0, 0
roberta.cuda()
roberta.eval()


RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-23): 24 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       

In [ ]:
def get_predictions(fout_path, fin_path):
  with open(fout_path, "w") as fout:
    with open(fin_path) as fin:
      for enumerator, row in enumerate(fin):
          row = json.loads(row)
          sent1, sent2, target = row["premise"], row["hypothesis"], row["label"]
          tokens = roberta.encode(sent1, sent2)
          try:
            prediction = roberta.predict('mnli', tokens).argmax().item()
          except ValueError:
            prediction = 3
          prediction_label = label_map[prediction]
          json.dump({"label": prediction_label}, fout)
          fout.write('\n')

          if enumerator % 400 == 0:
            print(enumerator)

In [ ]:
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_mismatched_bart_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_mismatched_bart.jsonl')
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_matched_bart_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_matched_bart.jsonl')

In [ ]:
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_mismatched_pegasus_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_mismatched_pegasus.jsonl')
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_matched_pegasus_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_matched_pegasus.jsonl')


In [ ]:
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_mismatched_gpt_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_mismatched_gpt.jsonl')
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_matched_gpt_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_matched_gpt.jsonl')


In [ ]:
# get_predictions('/content/drive/MyDrive/thesis/multinli_dev_mismatched_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/multinli_dev_mismatched.jsonl')
# get_predictions('/content/drive/MyDrive/thesis/multinli_dev_matched_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/multinli_dev_matched.jsonl')


In [ ]:
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_mismatched_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_mismatched.jsonl')
# get_predictions('/content/drive/MyDrive/thesis/paraNLI_matched_predictions.jsonl',
#                 '/content/drive/MyDrive/thesis/paraNLI_matched.jsonl')


In [ ]:
get_predictions('/content/drive/MyDrive/thesis/paraNLI_mismatched_annotations_predictions.jsonl',
                '/content/drive/MyDrive/thesis/paraNLI_mismatched_annotations.jsonl')
get_predictions('/content/drive/MyDrive/thesis/paraNLI_matched_annotations_predictions.jsonl',
                '/content/drive/MyDrive/thesis/paraNLI_matched_annotations.jsonl')

0
0
